In [1]:
import pandas as pd

In [60]:
abp_data = pd.read_csv('example_abp_data.csv')
input_data = pd.read_csv('example_input_data.csv')

In [4]:
print(abp_data.info(), input_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SINGLE_LINE_ADDRESS  2764 non-null   object
 1   POSTCODE             2764 non-null   object
 2   STREET_NAME          2520 non-null   object
dtypes: object(3)
memory usage: 64.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Address_Line_1  100 non-null    object
 1   Address_Line_2  90 non-null     object
 2   Address_Line_3  83 non-null     object
 3   Address_Line_4  45 non-null     object
 4   Address_Line_5  14 non-null     object
 5   Postcode        100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB
None None


In [6]:
abp_data.head()

,SINGLE_LINE_ADDRESS,POSTCODE,STREET_NAME
0,"PACEC, SECOND FLOOR, 49-53 REGENT STREET, CAMB...",CB2 1AB,REGENT STREET
1,"FIRST FLOOR, 47-53 REGENT STREET, CAMBRIDGE, C...",CB2 1AB,REGENT STREET
2,"BUBBLE LTD, SUITE 1, RAVENSCROFT HOUSE, 59-61 ...",CB2 1AB,REGENT STREET
3,"EDE & RAVENSCROFT, SUITE 2, RAVENSCROFT HOUSE,...",CB2 1AB,REGENT STREET
4,"CAMBRIDGE NUTRACEUTICALS LTD, SUITE 3, RAVENSC...",CB2 1AB,REGENT STREET


In [62]:
input_data.head()

,Address_Line_1,Address_Line_2,Address_Line_3,Address_Line_4,Address_Line_5,Postcode
0,UNIT 3,39-41 MOORBRIDGE ROAD,MAIDENHEAD,BERKSHIRE,NaN,SL6 8LT
1,4 WORKSHOP,PERCY ROAD,DARLINGTON,NaN,NaN,DL3 0RX
2,SAXON HOUSE,HENSON WAY,TELFORD WAY INDUSTRIAL ESTATE,KETTERING,NN16 8PX,NN16 8PX
3,1 QUARRY ROAD,NORTHAMPTON,NN5 6NT,NaN,NaN,NN5 6NT
4,METER R/O UNIT 54 UNKNOWN SUPPLY,DALTON PARK,MURTON,SEAHAM,NaN,SR7 9HU


In [71]:
def check_address(postcode, address_lines, abp_data):
    """
    Check if any of the address lines appear in the STREET_NAME or SINGLE_LINE_ADDRESS columns of abp_data for a given postcode.

    Parameters:
    postcode (str): The postcode to match in the abp_data DataFrame.
    address_lines (list): A list of address lines to check against the STREET_NAME and SINGLE_LINE_ADDRESS columns.
    abp_data (DataFrame): The DataFrame containing SINGLE_LINE_ADDRESS, POSTCODE, and STREET_NAME columns.

    Returns:
    str: 'Yes' if any address line appears in STREET_NAME for the same postcode, 'No' otherwise.
    """
    matching_rows = abp_data[abp_data['POSTCODE'] == postcode]
    for _, row in matching_rows.iterrows():
        street_name = row['STREET_NAME']
        full_address = row['SINGLE_LINE_ADDRESS']
        if pd.notna(street_name):  # Check if street_name is not NaN
            for line in address_lines:
                if (line.upper() in street_name.upper()) or (line.upper() in full_address.upper()): # Return 'Yes' if in either STREET_NAME or SINGLE_LINE_ADDRESS
                    return 'Yes'
        # If STREET_NAME is NaN but correct address is in SINGLE_LINE_ADDRESS return 'Yes'
        elif pd.isna(street_name):
            for line in address_lines:
                if line.upper() in full_address.upper():
                    return 'Yes'
    return 'No'

def create_street_in_postcode_col(input_data, abp_data):
    """
    Update the input_data DataFrame with a 'Street_In_Postcode' column indicating if any address lines appear in STREET_NAME or SINGLE_LINE_ADDRESS for the same postcode.

    Parameters:
    input_data (DataFrame): The DataFrame containing address lines and postcodes.
    abp_data (DataFrame): The DataFrame containing SINGLE_LINE_ADDRESS, POSTCODE, and STREET_NAME columns.

    Returns:
    DataFrame: The updated input_data DataFrame with the 'Street_In_Postcode' column.
    """
    input_data['Street_In_Postcode'] = input_data.apply(
        lambda row: check_address(
            row['Postcode'],
            [str(line) if pd.notna(line) else '' for line in [
            row['Address_Line_1'], row['Address_Line_2'], row['Address_Line_3'], row['Address_Line_4'], row['Address_Line_5']
        ]],
            abp_data
        ),
        axis=1
    )
    return input_data

Only issue with this implementation is that if only the town appears in the full address, then we return yes.

Also not sure if they want me to check just the STREET_NAME column or also the SINGLE_ADDRESS_LINE column.

In [74]:
input_data = create_street_in_postcode_col(input_data, abp_data)

In [73]:
input_data.head(60)

,Address_Line_1,Address_Line_2,Address_Line_3,Address_Line_4,Address_Line_5,Postcode,Street_In_Postcode
0,UNIT 3,39-41 MOORBRIDGE ROAD,MAIDENHEAD,BERKSHIRE,NaN,SL6 8LT,No
1,4 WORKSHOP,PERCY ROAD,DARLINGTON,NaN,NaN,DL3 0RX,Yes
2,SAXON HOUSE,HENSON WAY,TELFORD WAY INDUSTRIAL ESTATE,KETTERING,NN16 8PX,NN16 8PX,Yes
3,1 QUARRY ROAD,NORTHAMPTON,NN5 6NT,NaN,NaN,NN5 6NT,Yes
4,METER R/O UNIT 54 UNKNOWN SUPPLY,DALTON PARK,MURTON,SEAHAM,NaN,SR7 9HU,Yes
5,UNIT 17,MAITLAND ROAD,NEEDHAM MARKET,IPSWICH,IP6 8NZ,IP6 8NZ,Yes
6,NORTH MALTING THE MALTINGS,FELAW STREET,IPSWICH,IP2 8HD,NaN,IP2 8HD,Yes
7,19-21 REGENT STREET,NaN,NaN,NaN,CAMBRIDGE,CB2 1AB,Yes
8,UNIT 3 RIVERSIDE HOUSE,24 LOWER SOUTHEND ROAD,WICKFORD,NaN,NaN,SS11 8AW,Yes
9,WORKSHOP JOHNS AUTOS,TEMPERANCE PLACE,DARLINGTON,NaN,NaN,DL3 6LP,Yes


In [57]:
input_data.loc[input_data['Postcode'] == 'SL6 8LT']

,Address_Line_1,Address_Line_2,Address_Line_3,Address_Line_4,Address_Line_5,Postcode,Match
0,UNIT 3,39-41 MOORBRIDGE ROAD,MAIDENHEAD,BERKSHIRE,NaN,SL6 8LT,No


In [56]:
abp_data.loc[abp_data['POSTCODE'] == 'SL6 8LT']

,SINGLE_LINE_ADDRESS,POSTCODE,STREET_NAME
